# Day 16: Aunt Sue

[*Advent of Code 2015 day 16*](https://adventofcode.com/2015/day/16) and [*solution megathread*](https://www.reddit.com/3x1i26)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2015/16/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2015%2F16%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
%load_ext nb_mypy

Version 1.0.4


In [6]:
%nb_mypy On

## Comments

Funny we didn't get any test data this time - but this ought to be quite doable with a bit of Pandas?

In [7]:
from typing import Dict
from IPython.display import display


categories_str = """children
cats
samoyeds
pomeranians
akitas
vizslas
goldfish
trees
cars
perfumes""".splitlines()

traces_str = """children: 3
cats: 7
samoyeds: 2
pomeranians: 3
akitas: 0
vizslas: 0
goldfish: 5
trees: 3
cars: 2
perfumes: 1""".splitlines()

# This was created before we started nb_mypy
downloaded: Dict[str, str]
inputdata = downloaded['input'].splitlines()
display(f'{inputdata[0]} ...')

'Sue 1: cars: 9, akitas: 3, goldfish: 0 ...'

In [8]:
from typing import TypeVar, Any, Tuple, List


def parse_data(
        data: List[str],
        traces: List[str],
        categories: List[str]) -> Tuple[Dict[int, Dict[str, int]],
                                        Dict[str, int]]:
    K = TypeVar('K')

    def value_to_int(k: K, v: str) -> Tuple[K, int]:
        return k, int(v)
    output_data = dict()
    for line in data:
        label_s, items_s = line.split(': ', 1)
        label = int(label_s[len('Sue '):])
        output_data[label] = dict(value_to_int(*item_s.split(': ', 1))
                                  for item_s in items_s.split(', '))
        for key in output_data[label].keys():
            assert key in categories_str, f'{key=} not in {categories_str=}'
    output_traces = dict(value_to_int(*item_s.split(': ', 1))
                         for item_s in traces)
    for key in output_traces.keys():
        assert key in categories_str, f'{key=} not in {categories_str=}'
    return output_data, output_traces

In [9]:
data_dict, traces_dict = parse_data(inputdata, traces_str, categories_str)

In [10]:
eliminated_aunts = []
for substance, amount in traces_dict.items():
    for aunt_id, known_posessions in data_dict.items():
        if (
                substance in known_posessions.keys() and
                amount != known_posessions[substance]):
            eliminated_aunts.append(aunt_id)
            break
remaining_aunts = {aunt_id: known_posessions
                   for aunt_id, known_posessions in data_dict.items()
                   if aunt_id not in eliminated_aunts}
for aunt_id, known_posessions in remaining_aunts.items():
    for substance, amount in known_posessions.items():
        if (
                substance not in traces_dict.keys() or
                amount != traces_dict[substance]):
            eliminated_aunts.append(aunt_id)
            break
remaining_aunts = {aunt_id: known_posessions
                   for aunt_id, known_posessions in data_dict.items()
                   if aunt_id not in eliminated_aunts}
display(f'{remaining_aunts=}')

"remaining_aunts={373: {'pomeranians': 3, 'perfumes': 1, 'vizslas': 0}}"

In [11]:
# my_part1_solution(inputdata)

In [12]:
HTML(downloaded['part1_footer'])

<cell>1: error: Name "HTML" is not defined  [name-defined]


## Part Two

In [13]:
HTML(downloaded['part2'])

<cell>1: error: Name "HTML" is not defined  [name-defined]


In [14]:
# In particular, the cats and trees readings indicates that there are greater
# than that many (due to the unpredictable nuclear decay of cat dander and tree
# pollen), while the pomeranians and goldfish readings indicate that there are
# fewer than that many (due to the modial interaction of magnetoreluctance).
limits_greater = ['cats', 'trees']
limits_fewer = ['pomeranians', 'goldfish']

In [15]:
eliminated_aunts = []
for substance, amount in traces_dict.items():
    for aunt_id, known_posessions in data_dict.items():
        if (substance in known_posessions.keys() and (
                (substance in limits_greater and
                 amount > known_posessions[substance]) or
                (substance in limits_fewer and
                 amount < known_posessions[substance]) or
                amount != known_posessions[substance])):
            eliminated_aunts.append(aunt_id)
            # display(f'Eliminated aunt {aunt_id} due to having wrong amount {known_posessions[substance]} of {substance} (expected {amount})')
            break
remaining_aunts = {aunt_id: known_posessions
                   for aunt_id, known_posessions in data_dict.items()
                   if aunt_id not in eliminated_aunts}
for aunt_id, known_posessions in remaining_aunts.items():
    for substance, amount in known_posessions.items():
        if (
                substance not in traces_dict.keys() or
                (substance not in limits_greater and
                 amount > traces_dict[substance]) or
                (substance not in limits_fewer and
                 amount < traces_dict[substance])):
            eliminated_aunts.append(aunt_id)
            # display(f'Eliminated aunt {aunt_id} due to not finding traces of {substance} ({traces_dict[substance]}) (expected {amount})')
            break
remaining_aunts = {aunt_id: known_posessions
                   for aunt_id, known_posessions in data_dict.items()
                   if aunt_id not in eliminated_aunts}
display(f'{remaining_aunts=}')

"remaining_aunts={260: {'goldfish': 0, 'vizslas': 0, 'samoyeds': 2}, 373: {'pomeranians': 3, 'perfumes': 1, 'vizslas': 0}}"

11:80: E501 line too long (143 > 79 characters)
25:80: E501 line too long (139 > 79 characters)


In [16]:
# assert(my_part2_solution(testdata) == ...)

In [17]:
# my_part2_solution(inputdata)

In [18]:
HTML(downloaded['part2_footer'])

<cell>1: error: Name "HTML" is not defined  [name-defined]
